<a href="https://colab.research.google.com/github/HatemMoushir/Sentiment/blob/main/sent140_multilingual_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ✅ الخطوة 1: تثبيت المكتبات اللازمة
!pip install -q transformers datasets

# ✅ الخطوة 2: استيراد الأدوات
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import re

# ✅ الخطوة 3: تحميل عينة من بيانات Sentiment140 (1000 إنجليزية)
ds_en = load_dataset("sentiment140", split="train[:1000]")
ds_en = ds_en.rename_columns({"text": "text", "sentiment": "label"})

# تحويل labels إلى 0 و 1 فقط
ds_en = ds_en.filter(lambda example: example["label"] in [0, 4])
ds_en = ds_en.map(lambda x: {"label": 0 if x["label"] == 0 else 1})

# ✅ الخطوة 4: تحميل AraSenti-Tweet (كلها، ثم نأخذ 1000 عينة عشوائية)
ds_ar_full = load_dataset("ARBML/AraSenti-Tweet", split="train")
ds_ar = ds_ar_full.shuffle(seed=42).select(range(1000))
ds_ar = ds_ar.rename_columns({"Tweet": "text", "label": "label"})

# ✅ الخطوة 5: تنظيف التويتات (اختياري)
def clean_text(example):
    text = example["text"]
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+|[^\w\s]", '', text)  # remove links, mentions, hashtags, punctuations
    return {"text": text.strip().lower()}

ds_en = ds_en.map(clean_text)
ds_ar = ds_ar.map(clean_text)

# ✅ الخطوة 6: دمج الداتا
full_ds = concatenate_datasets([ds_en, ds_ar]).shuffle(seed=123)

# ✅ الخطوة 7: تحميل Tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_ds = full_ds.map(tokenize, batched=True)

# ✅ الخطوة 8: تقسيم التدريب والاختبار
split = tokenized_ds.train_test_split(test_size=0.1, seed=123)
train_ds = split["train"]
test_ds = split["test"]

# ✅ الخطوة 9: تحميل النموذج
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# ✅ الخطوة 10: إعدادات التدريب
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
)

# ✅ الخطوة 11: تعريف المدرب Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
)

# ✅ الخطوة 12: بدء التدريب
trainer.train()